In [1]:
# Import generic wrappers
from transformers import AutoModel, AutoTokenizer 


# Define the model repo
model_name = "DeepPavlov/rubert-base-cased" 


# Download pytorch model
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import numpy as np
import pymorphy2
import nltk
import joblib

stop_words=joblib.load(open("stop_words.joblib","rb"))
punctuation=joblib.load(open("punctuation.joblib","rb"))

In [3]:
#nltk.download()
#nltk.download('punkt')
#nltk.download('stopwords')
# from nltk.corpus import stopwords
# import nltk.data
# punctuation=[i.replace(" ","") for i in nltk.data.load("Punctuation.txt",format="text")]
# #stop_words=[i.replace(" ","") for i in nltk.data.load("..//..//nltk_data/corpora/stopwords/russian",format="text").split("\n")]
# ru_stop_words=stopwords.words('russian')
# en_stop_words=stopwords.words('english')
# stop_words=ru_stop_words+en_stop_words
# print(len(stop_words))
# import joblib
# joblib.dump(stop_words,open("stop_words.joblib","wb"))
# joblib.dump("".join(punctuation),open("punctuation.joblib","wb"))

In [4]:
def text_prepare(text):
    global stop_words
    text=text.lower()
    tokenized_str = word_tokenize(text)
    morph = pymorphy2.MorphAnalyzer()
    parsed_tokens = [morph.parse(word) for word in tokenized_str if word not in punctuation]
    normal_forms = [parsed_token[0].normal_form for parsed_token in parsed_tokens]
    tokens = np.array([i for i in normal_forms if i not in stop_words])
    return " ".join(tokens)

def get_text_vector(text):
    global tokenizer
    global model
    # Transform input tokens 
    inputs = tokenizer(text, return_tensors="pt")
    # Model apply
    outputs = model(**inputs)
    return outputs.pooler_output.detach().numpy()

def get_n_grams(raw_text: str,ngram_count: int):
    global punctuation
    text=text_prepare(raw_text)
    tokenized_str = word_tokenize(text)
    tokens = np.array([i for i in tokenized_str])    
    return np.array([" ".join(list(i)) for i in ngrams(tokens,ngram_count)])

In [5]:
text="""Жила девочка Женя. Однажды послала её мама в магазин за баранками. Купила Женя семь баранок: две баранки с тмином для папы, две баранки с маком для мамы, две баранки с сахаром для себя и одну маленькую розовую баранку для братика Павлика.

Взяла Женя связку баранок и отправилась домой. Идет, по сторонам зевает, вывески читает, ворон считает. А сзади тем временем сзади пристала незнакомая собака, да все баранки одну за другой и съела. Сначала съела папины с тмином, потом мамины с маком, потом Женины с сахаром. Почувствовала Женя, что баранки стали что-то чересчур легкие. Обернулась, да уж поздно. Мочалка болтается пустая, а собака последнюю, розовую Павликову бараночку доедает, и счастливо облизывается.

- Ах, вредная собака! - закричала Женя и бросилась ее догонять.

Бежала, бежала, собаку не догнала, только сама заблудилась. Видит – место совсем незнакомое, больших домов нет, а стоят маленькие домики. Испугалась Женя и заплакала."""

In [6]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

def get_key_words(text,count_output_words=5,max_n_grams=3):
    text=text_prepare(text)
    all_text_vector=get_text_vector(text)
    # preparation of vectorizable n-grams
    grams=np.array([])
    for i in range(1,max_n_grams+1):
        grams=np.concatenate((grams,np.unique(get_n_grams(text,i))))
    # their vectorization
    vectors=np.array(list(map(get_text_vector,grams)))
    vectors=vectors.reshape(vectors.shape[0],vectors.shape[-1])
    # calculating the cosine distance
    cos_dist=np.array(list(map(lambda x: cosine_distances(x.reshape(1,-1),all_text_vector.reshape(1,-1)),vectors)))
    cos_dist=cos_dist.reshape(-1,)
    # dictionary formation with cosine distances
    dist2gram={cos_dist[i]:grams[i] for i in range(len(cos_dist))}
    mins_dist=np.sort(cos_dist)[:count_output_words]
    return np.array(list(map(lambda x: {dist2gram[x]:x},mins_dist)))

In [7]:
get_key_words(text,10,2)

array([{'жить': 0.061053634}, {'дом стоять': 0.06236291},
       {'папин тмин': 0.06507236}, {'папа': 0.06588364},
       {'домик': 0.0678969}, {'папин': 0.07127875},
       {'мамин мак': 0.071457386}, {'мак мама': 0.07146311},
       {'сахар': 0.07181221}, {'дом': 0.07377851}], dtype=object)